# 1. Data Viewing

In [ ]:
import pandas as pd
import os
from IPython.display import display

data_dir = "../data/Train"

In [ ]:
# Load train_terms.tsv
print("Loading train_terms.tsv...")
train_terms_path = os.path.join(data_dir, "train_terms.tsv")
train_terms_df = pd.read_csv(train_terms_path, sep="\t")
print("train_terms.tsv head:")
display(train_terms_df.head())


In [ ]:



train_sequences_path = os.path.join(data_dir, "train_sequences.fasta")

sequences = []
current_sequence_id = None
current_sequence = []
current_header = None

def parse_header(header_line):
    """
    Input: header_line: string without leading '>'
    Return: dict with parsed fields:
      - header: full header (raw)
      - db_source, accession, entry_name, description, OS, OX, GN, PE, SV
    """
    header = header_line.strip()
    result = {'header': header,
              'db_source': None, 'accession': None, 'entry_name': None,
              'description': None, 'OS': None, 'OX': None, 'GN': None, 'PE': None, 'SV': None}

    # primary token (something like sp|A0JNW5|BLT3B_HUMAN)
    parts = header.split(' ', 1)
    primary = parts[0]
    rest = parts[1] if len(parts) > 1 else ''
    primary_parts = primary.split('|')
    if len(primary_parts) >= 3:
        result['db_source'] = primary_parts[0]
        result['accession'] = primary_parts[1]
        result['entry_name'] = primary_parts[2]
    else:
        # fallback: set whole primary into entry_name
        result['entry_name'] = primary

    # description is the text before the first " OS=" token (if present)
    if ' OS=' in rest:
        desc, fields_str = rest.split(' OS=', 1)
        result['description'] = desc.strip()
        fields_str = 'OS=' + fields_str  # restore the OS= for parsing
    else:
        result['description'] = rest.strip()
        fields_str = ''

    # normalize delimiters then split into key=value tokens
    if fields_str:
        # ensure consistent separators for known keys
        for key in [' OX=', ' GN=', ' PE=', ' SV=']:
            fields_str = fields_str.replace(key, '|' + key.strip())
        # Also prefix OS= if it wasn't prefixed by '|'
        fields_str = fields_str.replace('OS=', 'OS=').lstrip('|')
        for token in fields_str.split('|'):
            if '=' not in token:
                continue
            k, v = token.split('=', 1)
            k = k.strip()
            v = v.strip()
            if k in result:
                result[k] = v
            else:
                # assign to known ones explicitly
                if k == 'OS':
                    result['OS'] = v
                elif k == 'OX':
                    result['OX'] = v
                elif k == 'GN':
                    result['GN'] = v
                elif k == 'PE':
                    result['PE'] = v
                elif k == 'SV':
                    result['SV'] = v
    return result

# read file safely
try:
    with open(train_sequences_path, 'r') as f:
        for line in f:
            line = line.rstrip('\n')
            if not line:
                continue
            if line.startswith('>'):
                # save previous sequence
                if current_sequence_id is not None:
                    seq_str = ''.join(current_sequence)
                    row = {
                        'sequence_id': current_sequence_id,
                        'sequence': seq_str,
                        'length': len(seq_str),
                    }
                    # attach parsed header fields
                    row.update(current_header)
                    sequences.append(row)

                # start new record
                header_line = line[1:].strip()
                current_header = parse_header(header_line)
                current_sequence_id = current_header.get('accession') or current_header.get('entry_name') or header_line
                current_sequence = []
            else:
                # sequence lines: remove whitespace and append
                current_sequence.append(line.strip())

        # final record
        if current_sequence_id is not None:
            seq_str = ''.join(current_sequence)
            row = {
                'sequence_id': current_sequence_id,
                'sequence': seq_str,
                'length': len(seq_str),
            }
            row.update(current_header)
            sequences.append(row)

except FileNotFoundError:
    raise FileNotFoundError(f"Không tìm thấy file: {train_sequences_path}. Kiểm tra lại data_dir hoặc tên file.")

# build dataframe and display
train_sequences_df = pd.DataFrame(sequences)

# reorder columns for nicer view (if present)
preferred_cols = ['sequence_id', 'accession', 'entry_name', 'db_source', 'description',
                  'OS', 'OX', 'GN', 'PE', 'SV', 'length', 'sequence', 'header'] 
cols = [c for c in preferred_cols if c in train_sequences_df.columns] + \
       [c for c in train_sequences_df.columns if c not in preferred_cols]

train_sequences_df = train_sequences_df[cols]

print("train_sequences.fasta head:")
display(train_sequences_df.head())
print("\nTotal sequences loaded:", len(train_sequences_df))



In [ ]:
# Load train_taxonomy.tsv
print("Loading train_taxonomy.tsv...")
train_taxonomy_path = os.path.join(data_dir, "train_taxonomy.tsv")
train_taxonomy_df = pd.read_csv(train_taxonomy_path, sep="\t")
print("train_taxonomy.tsv head:")
display(train_taxonomy_df.head())
print("\n")


In [ ]:
# Load go-basic.obo into a DataFrame
print("Loading go-basic.obo...")
go_obo_path = os.path.join(data_dir, "go-basic.obo")
terms = []
current_term = {}

with open(go_obo_path, 'r') as f:
    for line in f:
        line = line.strip()
        if line == '[Term]':
            if current_term:
                terms.append(current_term)
            current_term = {}
        elif line.startswith('id:'):
            current_term['id'] = line.split(':', 1)[1].strip()
        elif line.startswith('name:'):
            current_term['name'] = line.split(':', 1)[1].strip()
        elif line.startswith('namespace:'):
            current_term['namespace'] = line.split(':', 1)[1].strip()
        elif line.startswith('def:'):
            current_term['def'] = line.split(':', 1)[1].strip().strip('"')
        elif line.startswith('is_a:'):
            # Extract only the GO ID, ignore the name after '!'
            is_a_id = line.split(':', 1)[1].split('!', 1)[0].strip()
            if 'is_a' not in current_term:
                current_term['is_a'] = []
            current_term['is_a'].append(is_a_id)
    if current_term:  # Add the last term
        terms.append(current_term)

go_obo_df = pd.DataFrame(terms)
print("go-basic.obo head:")
display(go_obo_df.head())
print("\n")


In [ ]:
print("Data loading complete.")
print("train_sequences_df columns:", train_sequences_df.columns)
print("train_taxonomy_df columns:", train_taxonomy_df.columns)
print("go_obo_df columns:", go_obo_df.columns)
print("train_terms_df columns:", train_terms_df.columns)

In [ ]:
import os, json, warnings, time
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, average_precision_score
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

In [ ]:
warnings.filterwarnings('ignore')
print("PyTorch version:", torch.__version__)

In [ ]:
# TF-IDF limits to avoid huge dense tensors on GPU
TFIDF_MAX_FEATURES = 50000   # lower if memory is tight
BATCH_SIZE = 256
EPOCHS = 8
LR = 1e-3
WEIGHT_DECAY = 1e-5
DROPOUT = 0.3
HIDDEN_DIM = 1024
RANDOM_STATE = 42


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


In [ ]:
# ---------- 0. Load precomputed resources (assume train_terms_df and train_sequences_df exist) ----------
# If not present, load from files (adapt paths as needed)
if 'train_terms_df' not in globals():
    train_terms_df = pd.read_csv(os.path.join(TRAIN_DIR, 'train_terms.tsv'), sep='\t')
if 'train_sequences_df' not in globals():
    # use your existing parse function or data already loaded in session
    raise RuntimeError("train_sequences_df not found in globals. Load it first.")

# map sequences: sequence_id -> sequence
seq_map = dict(zip(train_sequences_df['sequence_id'], train_sequences_df['sequence']))
all_entry_ids = set(train_terms_df['EntryID'].unique()).intersection(seq_map.keys())
entries = sorted(all_entry_ids)
seqs = [seq_map[e] for e in entries]
print(f"Train proteins with sequences: {len(entries)}")

In [ ]:
# ---------- 1. TF-IDF features ----------
vectorizer = TfidfVectorizer(analyzer='char', ngram_range=(3,3), min_df=2, max_features=TFIDF_MAX_FEATURES)
X_sparse = vectorizer.fit_transform(seqs)
print("TF-IDF shape (sparse):", X_sparse.shape)

# ---------- 2. Build multi-label Y (we'll train per-aspect; here show MF/CC/BP loop as before) ----------
aspect_names = {'F': 'MF', 'P': 'BP', 'C': 'CC'}
models_summary = {}
def eval_best_threshold(y_true, y_prob, thresholds=np.linspace(0.1,0.9,17)):
    best_f1, best_t = 0.0, 0.5
    for t in thresholds:
        y_pred = (y_prob >= t).astype(int)
        f1 = f1_score(y_true, y_pred, average='micro', zero_division=0)
        if f1 > best_f1:
            best_f1, best_t = f1, t
    return best_t, best_f1

In [ ]:
# ---------- 3. loop aspects and train PyTorch MLP ----------
for aspect_code, aspect_label in aspect_names.items():
    print("\n" + "="*60)
    print(f"Training aspect {aspect_label} ({aspect_code})")
    # gather labels for proteins in 'entries' order
    labels_map = train_terms_df[train_terms_df['aspect']==aspect_code].groupby('EntryID')['term'].apply(list).to_dict()
    y_labels = [labels_map.get(e, []) for e in entries]
    mlb = MultiLabelBinarizer()
    Y = mlb.fit_transform(y_labels)
    n_labels = Y.shape[1]
    print("Number of labels (terms):", n_labels)
    if n_labels == 0:
        print("No labels for this aspect; skipping.")
        continue

    # train/val split
    X_train_idx, X_val_idx, Y_train, Y_val = train_test_split(
        np.arange(len(entries)), Y, test_size=0.2, random_state=RANDOM_STATE)
    # convert sparse -> dense float32 (be careful with memory). If too big, reduce TFIDF_MAX_FEATURES.
    X = X_sparse.tocsr()
    X_train = X[X_train_idx].toarray().astype(np.float32)
    X_val = X[X_val_idx].toarray().astype(np.float32)
    print("Dense shapes:", X_train.shape, X_val.shape)

    # Build datasets and dataloaders
    train_ds = TensorDataset(torch.from_numpy(X_train), torch.from_numpy(Y_train.astype(np.float32)))
    val_ds = TensorDataset(torch.from_numpy(X_val), torch.from_numpy(Y_val.astype(np.float32)))
    train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, drop_last=False)
    val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False)

    # Simple MLP
    class MLP(nn.Module):
        def __init__(self, input_dim, hidden_dim, out_dim, dropout=0.3):
            super().__init__()
            self.net = nn.Sequential(
                nn.Linear(input_dim, hidden_dim),
                nn.BatchNorm1d(hidden_dim),
                nn.ReLU(),
                nn.Dropout(dropout),
                nn.Linear(hidden_dim, hidden_dim//2),
                nn.BatchNorm1d(hidden_dim//2),
                nn.ReLU(),
                nn.Dropout(dropout),
                nn.Linear(hidden_dim//2, out_dim)
            )
        def forward(self, x):
            return self.net(x)

    model = MLP(input_dim=X_train.shape[1], hidden_dim=HIDDEN_DIM, out_dim=n_labels, dropout=DROPOUT).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
    criterion = nn.BCEWithLogitsLoss()

    # training loop with tqdm
    best_val_f1 = -1.0
    best_state = None
    for epoch in range(1, EPOCHS+1):
        model.train()
        running_loss = 0.0
        pbar = tqdm(train_loader, desc=f"Epoch {epoch}/{EPOCHS} train", leave=False)
        for xb, yb in pbar:
            xb = xb.to(device); yb = yb.to(device)
            optimizer.zero_grad()
            logits = model(xb)
            loss = criterion(logits, yb)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * xb.size(0)
            pbar.set_postfix({'loss': f"{loss.item():.4f}"})
        epoch_loss = running_loss / len(train_loader.dataset)

        # validation
        model.eval()
        val_loss = 0.0
        all_probs = []
        all_trues = []
        with torch.no_grad():
            for xb, yb in val_loader:
                xb = xb.to(device); yb = yb.to(device)
                logits = model(xb)
                loss = criterion(logits, yb)
                val_loss += loss.item() * xb.size(0)
                probs = torch.sigmoid(logits).cpu().numpy()
                all_probs.append(probs)
                all_trues.append(yb.cpu().numpy())
        val_loss /= len(val_loader.dataset)
        all_probs = np.vstack(all_probs)
        all_trues = np.vstack(all_trues)

        # compute micro F1 at default threshold 0.5 and also best threshold
        f1_05 = f1_score(all_trues, (all_probs>=0.5).astype(int), average='micro', zero_division=0)
        best_t, best_f1 = eval_best_threshold(all_trues, all_probs, thresholds=np.linspace(0.1,0.9,17))

        print(f"Epoch {epoch:02d} | train_loss={epoch_loss:.4f} val_loss={val_loss:.4f} val_f1@0.5={f1_05:.4f} best_f1={best_f1:.4f} (t={best_t:.2f})")

        # save best
        if best_f1 > best_val_f1:
            best_val_f1 = best_f1
            best_state = {
                'model_state': model.state_dict(),
                'mlb_classes': list(mlb.classes_),
                'vectorizer_vocab_size': X_train.shape[1],
                'best_threshold': float(best_t),
                'val_f1': float(best_f1)
            }

    # after epochs: restore best state and save model
    if best_state is not None:
        model.load_state_dict(best_state['model_state'])
        # Save model checkpoint (PyTorch)
        save_dir = os.path.join(BASE_DIR, 'models')
        os.makedirs(save_dir, exist_ok=True)
        fname = f"mlp_aspect_{aspect_label}.pth"
        torch.save({
            'model_state': model.state_dict(),
            'mlb_classes': best_state['mlb_classes'],
            'vectorizer': vectorizer,   # caution: may be large; you can save separately via pickle
            'best_threshold': best_state['best_threshold'],
            'val_f1': best_state['val_f1']
        }, os.path.join(save_dir, fname))
        print(f"Saved best model for {aspect_label} to {os.path.join(save_dir, fname)} (val_f1={best_state['val_f1']:.4f})")

    models_summary[aspect_code] = {
        'n_labels': n_labels,
        'best_val_f1': float(best_val_f1),
        'best_threshold': float(best_state['best_threshold']) if best_state is not None else None,
        'model_path': os.path.join(save_dir, fname) if best_state is not None else None
    }

print("\nTraining summary:")
print(json.dumps(models_summary, indent=2))
